# LangChain + GitHub Models + Azure OpenAI

LangChain is a modular framework designed to streamline the development and integration of applications utilizing large language models (LLMs) by providing tools for effective prompt engineering, data handling, and complex workflows.

LangChain supports a number of model providers including Azure OpenAI and Cohere. Below find examples of how LangChain can be used with these models provided by the GitHub Models service.


## 1. Install dependencies

In [1]:
%pip install langchain-openai
%pip install langchain-community
%pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.0/763.0 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: openai
    Found existing installation: openai 1.37.2
    Uninstalling openai-1.37.2:
      Successfully uninstalled openai-1.37.2
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 38.0 MB/s 

## 2. Constructing LangChain's ChatOpenAI class

Setting up the OpenAI API key and base URL for LangChain is the same as for OpenAI's Python client. You can set the `OPENAI_API_KEY` and `OPENAI_API_BASE` environment variables, or pass them directly to the `ChatOpenAI` class.


In [1]:
from langchain_openai import ChatOpenAI
import dotenv
import os

dotenv.load_dotenv()

if not os.getenv("GITHUB_TOKEN"):
    raise ValueError("GITHUB_TOKEN is not set")

os.environ["OPENAI_API_KEY"] = os.getenv("GITHUB_TOKEN")
os.environ["OPENAI_BASE_URL"] = "https://models.inference.ai.azure.com/"

GPT_MODEL = "gpt-4o-mini"

llm = ChatOpenAI(model=GPT_MODEL)

Now you can use the ChatOpenAI class to interact with the OpenAI API. 

In [2]:
from IPython.display import Markdown, display

response = llm.invoke("What is the meaning of life?")
display(Markdown(response.content))

The meaning of life is a deeply philosophical question that varies for each individual. Many people find meaning through relationships, personal growth, contributing to society, pursuing passions, or seeking spiritual fulfillment. Some philosophical perspectives suggest that meaning is something we create for ourselves, while others propose that it is inherent in existence. Ultimately, it often comes down to personal beliefs, experiences, and values. What resonates most with you?

## 3. Use a prompt template

LangChain has the concept of a prompt template that allows you parameterize the prompt and fill in the parameters with values. 

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a fun assistant that uses a lot of emojis."),
    ("user", "{usertext}")
])

Next, we combine the promt with the ChatOpenAI into a chain:

In [4]:
chain = prompt | llm 

Then we invoke the chain with the input data:

In [5]:
chain.invoke({"usertext": "when was the first ESC held?"})


AIMessage(content='The first Eurovision Song Contest (ESC) was held on May 24, 1956! 🎤🌍✨ It took place in Lugano, Switzerland, and featured seven countries competing with two songs each. What a musical journey it has been since then! 🎶🌈', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 30, 'total_tokens': 87}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_80a1bad4c7', 'finish_reason': 'stop', 'logprobs': None}, id='run-a2455656-dbb9-4c2a-9fb9-418c736dacc6-0', usage_metadata={'input_tokens': 30, 'output_tokens': 57, 'total_tokens': 87})

## 4. Add a parsing step to your chain

We can also extend the chain to include additional steps, such as parsing the content from the reponse:

In [8]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = prompt | llm | output_parser

chain.invoke({"usertext": "when was the first ESC held?"})

'The first Eurovision Song Contest (ESC) was held on May 24, 1956! 🎤🌍 It took place in Lugano, Switzerland, and featured just seven countries. Can you believe it? 🎶✨'

## 5. Use retrieval to ground the model

Now let's ask for some more recent information:

In [9]:
chain.invoke({"usertext": f"Who won the 2024 ESC?"})

"I don't have the latest info on the 2024 Eurovision Song Contest yet! 🎤✨ But you can check out the official Eurovision website or social media for the most up-to-date results! 🌍🎶 Who were you rooting for? 😊💖"

Looks like the model is not up to date with the latest contest. Let's help it out with some content from the wikipedia:

In [12]:
!pip install BeautifulSoup4

In [13]:
from langchain_community.document_loaders import WebBaseLoader

os.environ["USER_AGENT"] = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"

loader = WebBaseLoader("https://en.wikipedia.org/wiki/Eurovision_Song_Contest_2024")

docs = loader.load()
len(docs[0].page_content)

174114

Due to the limits of the free service, the content retrieved from Wikipedia is too large to be added to the context -- that is ok since we can use a vector store to index the content. For that we need an embedding model which we can use with the OpenAIEmbeddings class. Note that we need to reduce the chunk size due to the token limits of the GitHub Models free service. 

In [14]:
from langchain_openai import OpenAIEmbeddings

EMBEDDINGS_MODEL = "text-embedding-3-small"

# need to constrain the chunk_size to work within the limits of the GitHub Model service
embeddings = OpenAIEmbeddings(model=EMBEDDINGS_MODEL,chunk_size=30)

Now, we will use the [FAISS](https://github.com/facebookresearch/faiss) library to index the vectors for a similarity search.

In [15]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Next, we can create a document chain that will use some context and the user's question to provide an answer:

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""You are a fun assistant that uses a lot of emojis.
Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

We could just provide the context directly to the chain like so:

In [17]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "Who won 2024 ESC?",
    "context": [Document(page_content="The top 3 placings in the 2024 Eurovision Song Contest were:\n1. Switzerland\n2. Croatia\n3. Ukraine")]
})

'The winner of the 2024 Eurovision Song Contest was Switzerland! 🇨🇭🎶✨'

But we really want a retrieval chain that will retrieve the right documents from our vector index and then uses it as context in our prompt like so:

In [18]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

Let's ask this one who won the 2024 ESC:

In [19]:
response = retrieval_chain.invoke({"input": f"Who won the 2024 ESC?"})
print(response["answer"])


🎉 Switzerland won the Eurovision Song Contest 2024 with the song "The Code" performed by Nemo! 🇨🇭✨🎶


And a few more increasingly detailed questions:

In [20]:
response = retrieval_chain.invoke({"input": "where did the 2024 ESC take place?"})
print(response["answer"])

The 2024 Eurovision Song Contest took place in Malmö, Sweden! 🇸🇪🎤✨


In [21]:
response = retrieval_chain.invoke({"input": "who designed the stage for the 2024 ESC?"})
print(response["answer"])

The stage for the 2024 Eurovision Song Contest was designed by Florian Wieder! 🎤✨ He has designed the sets for six previous contests, making him quite the pro in the Eurovision world! 🎉🌈


In [22]:
response = retrieval_chain.invoke({"input": "when did the semi-finals for the 2024 ESC take place?"})
print(response["answer"])

The semi-finals for the Eurovision Song Contest 2024 took place on **7 May 2024** and **9 May 2024**! 🎤🌟🎶
